# Olist Cleaning and Checking Data

### Introduction

In this lesson we'll work with data from the [Olist Kaggle Competition](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce).

### Loading our data

We can list all of the data in a given directory with the following.

Which would output the below list.

In [3]:
datasets = ["olist_customers_dataset.csv", "olist_orders_dataset.csv",
"olist_geolocation_dataset.csv", "olist_products_dataset.csv",
"olist_order_items_dataset.csv", "olist_sellers_dataset.csv",
"olist_order_payments_dataset.csv", "product_category_name_translation.csv",
"olist_order_reviews_dataset.csv"]

And then, we can loop through our datasets, adding a key as the name of the dataset, and as a value storing the dataframe.

> First install the s3fs library.

In [7]:
# !pip3 install s3fs

Then we can read our datasets from s3.

In [8]:
import os
import pandas as pd
root_url = "s3://jigsaw-labs-student"
dataframe_set = {}

for csv_file in datasets:
    names = csv_file.split('_')[1:-1]
    full_name = '_'.join(names)
    dataframe_set[full_name] = pd.read_csv(f'{root_url}/olist-data/{csv_file}')

In [9]:
dataframe_set.keys()

dict_keys(['customers', 'orders', 'geolocation', 'products', 'order_items', 'sellers', 'order_payments', 'category_name', 'order_reviews'])

So above, we created a dictionary, where the keys are the names of the datasets and the values are the actual dataframes.

> For example, if we want to get the geolocation dataframe, we can with the following.

In [10]:
dataframe_set['geolocation'][:2]

geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0                         1037       -23.545621       -46.639292   
1                         1046       -23.546081       -46.644820   

  geolocation_city geolocation_state  
0        sao paulo                SP  
1        sao paulo                SP

It would be a lot to work with all of our datasets at once -- so looking at our data, what do you think are some of the key files that we should work with?

In [20]:
print(datasets)

['olist_customers_dataset.csv', 'olist_orders_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_products_dataset.csv', 'olist_order_items_dataset.csv', 'olist_sellers_dataset.csv', 'olist_order_payments_dataset.csv', 'product_category_name_translation.csv', 'olist_order_reviews_dataset.csv']


So we'll let you give it a shot, what are some of the main csv files to combine and work with.  Write down some below.

Then we'll move on.

### Combining our data

To identify our key csv files, we should probably first identify our join table and then work from there.  So above, a good our join table is probably `orders`.  Remember that our join table often involves our transactions, or events.  It turns out each order is more like a shopping cart, with many `order_items`.  So we should also use that dataset.  

From there, you want to make sure you have the various dimensions -- just ask the W questions as a starting point -- who, what, where, when.

Doing so, we'll join the `orders`, `order_items`, `customers` and `products` (sellers could also be added).  

In [22]:
orders_df = dataframe_set['orders']
orders_customers_df = orders_df.merge(dataframe_set['customers'], on = 'customer_id')
orders_products_df = orders_customers_df.merge(dataframe_set['order_items'], on = 'order_id').merge(dataframe_set['products'], on = 'product_id')
# dataframe_set['customers']

orders_products_df[:2]

order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  128e10d95713541c87cd1a2e48201934  a20e8105f23924cd00833fd87daa0831   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2017-08-15 18:29:31  2017-08-15 20:05:16   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2017-08-17 15:28:33           2017-08-18 14:44:43   

  order_estimated_delivery_date                customer_unique_id  \
0           2017-10-18 00:00:00  7c396fd4830fd04220f754e42b4e5bff   
1           2017-08-28 00:00:00  3a51803cc0d012c3b5dc8b7528cb05f7   

   customer_zip_code_prefix  ...  price freight_value  product_category_name  \
0                      3149  ...  29.99          8.72  utilidades_domesticas   
1                      3366  ...  29.99          7.78  utilidades_domesticas   

  product_name_lenght product_description_lenght product_photos_qty  \
0                40.0                      268.0                4.0   
1                40.0                      268.0                4.0   

   product_weight_g  product_length_cm product_height_cm  product_width_cm  
0             500.0               19.0               8.0              13.0  
1             500.0               19.0               8.0              13.0  

[2 rows x 26 columns]

An initial look at the data.

1. Understanding the grain of the data

* What is the grain of the data.  By this, we mean, what does each row represent?

In [70]:
# write the grain of the data here

2. Identifying the target of the data

In [ ]:
# write the target of the data here

### Cleaning the data

Now before cleaning the data, we can probably remove some of our columns. Currently, we have 26 columns from merging our three dataframes. 

In [31]:
orders_products_df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

> Let's just select the relevant columns below.  Assign each variable to a list of relevant columns (we chose nine in total).

In [71]:
target_cols = []

id_cols = []
order_cols = []
customer_cols = []
product_cols = []

selected_cols = order_cols + customer_cols + product_cols + id_cols + target_cols
print(selected_cols)

[]


Then from there we can select a subset of our data.

In [37]:
selected_df = orders_products_df[selected_cols]
selected_df[:2]

order_purchase_timestamp  customer_zip_code_prefix customer_city  \
0      2017-10-02 10:56:33                      3149     sao paulo   
1      2017-08-15 18:29:31                      3366     sao paulo   

  customer_state  product_category_name                          order_id  \
0             SP  utilidades_domesticas  e481f51cbdc54678b7cc49136f2d6af7   
1             SP  utilidades_domesticas  128e10d95713541c87cd1a2e48201934   

                        customer_id                        product_id  price  
0  9ef432eb6251297304e76186b10a928d  87285b34884572647811a353c7ac498a  29.99  
1  a20e8105f23924cd00833fd87daa0831  87285b34884572647811a353c7ac498a  29.99

> Ok, let's just show you the columns we chose.

In [40]:
target_cols = ['price']

id_cols = ['order_id', 'customer_id', 'product_id']
order_cols = [ 'order_purchase_timestamp']
customer_cols = ['customer_zip_code_prefix', 'customer_city', 'customer_state']
product_cols = ['product_category_name']

selected_cols = order_cols + customer_cols + product_cols + id_cols + target_cols
print(selected_cols)

['order_purchase_timestamp', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'product_category_name', 'order_id', 'customer_id', 'product_id', 'price']


> And then we can select just those columns, and rename one of the columns.

In [39]:
orders_products_df[selected_cols]

updated_selected_df = selected_df.rename(columns = {'order_purchase_timestamp': 'purchase_time'})
updated_selected_df[:2]

purchase_time  customer_zip_code_prefix customer_city customer_state  \
0  2017-10-02 10:56:33                      3149     sao paulo             SP   
1  2017-08-15 18:29:31                      3366     sao paulo             SP   

   product_category_name                          order_id  \
0  utilidades_domesticas  e481f51cbdc54678b7cc49136f2d6af7   
1  utilidades_domesticas  128e10d95713541c87cd1a2e48201934   

                        customer_id                        product_id  price  
0  9ef432eb6251297304e76186b10a928d  87285b34884572647811a353c7ac498a  29.99  
1  a20e8105f23924cd00833fd87daa0831  87285b34884572647811a353c7ac498a  29.99

#### 2. Cleaning our data

Now that we have merged together our datasets (by looking at the join table and W questions), identified the target and grain of the data, and then selected (pretty liberally) our relevant columns, it's time to clean our data.

1. Confirming the correct datatype

First, ensure that the columns are of the correct datatype.

In [36]:


# purchase_time                object
# customer_zip_code_prefix      int64
# customer_city                object
# customer_state               object
# product_category_name        object
# order_id                     object
# customer_id                  object
# product_id                   object
# price                       float64
# dtype: object

purchase_time                object
customer_zip_code_prefix      int64
customer_city                object
customer_state               object
product_category_name        object
order_id                     object
customer_id                  object
product_id                   object
price                       float64
dtype: object

Remember that we are essentially looking for columns that we should change from object to something else.  

> Above, these look pretty good, except for our `purchase_time` column, which we should change to be datetime.

> The `id` columns could being changed to be numeric -- however there doesn't appear to be much downside to leaving them as is.

2. Extracting datepart columns

Ok, so next use the `add_datepart` function to extract additional columns.  Add these columns to the copied dataframe (`selected_w_dates_df`). 

In [46]:
from item_lib import *

In [47]:
import warnings
warnings.filterwarnings('ignore')

selected_w_dates_df = updated_selected_df.copy()



Afterwards, we should see the following columns:

In [72]:
selected_w_dates_df.columns

# ['customer_zip_code_prefix', 'customer_city', 'customer_state',
#        'product_category_name', 'order_id', 'customer_id', 'product_id',
#        'price', 'purchase_timeYear', 'purchase_timeMonth', 'purchase_timeWeek',
#        'purchase_timeDay', 'purchase_timeDayofweek', 'purchase_timeDayofyear',
#        'purchase_timeElapsed']

Index(['customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_category_name', 'order_id', 'customer_id', 'product_id',
       'price', 'purchase_timeYear', 'purchase_timeMonth', 'purchase_timeWeek',
       'purchase_timeDay', 'purchase_timeDayofweek', 'purchase_timeDayofyear',
       'purchase_timeElapsed'],
      dtype='object')

### 3. Checking the representativeness of our data

* Selecting the relevant columns

So now we have a new set of relevant columns, which includes our date columns.  We can see these date columns at the end.

In [49]:
selected_w_dates_df.columns[8:]

Index(['purchase_timeYear', 'purchase_timeMonth', 'purchase_timeWeek',
       'purchase_timeDay', 'purchase_timeDayofweek', 'purchase_timeDayofyear',
       'purchase_timeElapsed'],
      dtype='object')

In [50]:
target_cols = ['price']

id_cols = ['order_id', 'customer_id', 'product_id']

customer_cols = ['customer_zip_code_prefix', 'customer_city', 'customer_state']
product_cols = ['product_category_name']
purchase_time_cols = ['purchase_timeYear', 'purchase_timeMonth', 'purchase_timeWeek',
       'purchase_timeDay', 'purchase_timeDayofweek', 'purchase_timeDayofyear',
       'purchase_timeElapsed']

selected_cols = customer_cols + product_cols + id_cols + purchase_time_cols + target_cols

From here, let's look at the representativeness of the data.  

As always, there are two main things to focus on here -- the time columns, and the target.  

### A. Exploring the time columns

* In this section, please spend a good amount of effort (1) exploring the time columns and (2) then just selecting a range of data that will allow us to explore a representative dataset going forward.

### 2. Exploring the target

Ok, next let's explore the target.  We want to check for missing data, or values that could be incorrect.

### Saving the data

* Finally, we can save our selected, and coerced dataset to a csv file so that we can use it going forward.

In [73]:
# selected_full_year_df.to_csv('./olist_2017_2018.csv', index = False)

### Takeaways

Ok, so what can we takeaway from the above.  

1. Selecting and merging data

Our first step was to load all of our data, identify the join table, add in our dimension tables (who, what, where, and when).

In [ ]:
orders_df = dataframe_set['orders']
orders_customers_df = orders_df.merge(dataframe_set['customers'], on = 'customer_id')
orders_products_df = orders_customers_df.merge(dataframe_set['order_items'], on = 'order_id').merge(dataframe_set['products'], on = 'product_id')

2. Initial identification

* Of the grain of the data (here it was individual order items)
* Of the target (here it was the price)

2. Initial cleanup

We select relevant columns, checked the datatypes, and used the `add_datepart` function to break the purchase datetime into multiple attributes.

3. Checking the representativeness of the data

* Time periods



* Target



In the next lesson, we have a write up of our steps for checking the representativeness of the data.